# Importing Necessary Librarires

In [61]:
import os
import pandas as pd
from shutil import copyfile
import numpy as np
import math
import cv2
import decimal
from pyproj import Proj, transform
from PIL import Image
from sympy import symbols, Eq, solve

# Defining Functions

In [2]:
def find_lat_lon(img1, img2):

    a = df_book2.loc[df_book2['ns1:ImageId'] == img1]
    lat1 = a.iloc[0]['ns1:Latitude']
    lon1 = a.iloc[0]['ns1:Longitude']

    b = df_book2.loc[df_book2['ns1:ImageId'] == img2]
    lat2 = b.iloc[0]['ns1:Latitude']
    lon2 = b.iloc[0]['ns1:Longitude']

    
    #print('Latitude and Longitude ----', lat1, lon1, lat2, lon2)
    return lat1, lon1, lat2, lon2

def get_names(img_name1, img_name2):

    or_name1 = os.path.splitext(img_name1)[0]
    or_name2 = os.path.splitext(img_name2)[0]

    f_name1 = or_name1 + '.txt'
    f_name2 = or_name2 + '.txt'

    base_path = r'D:\Distance Measurement\GB71_experimental_data'

    text_file_path_l = os.path.join(base_path, f_name1)
    text_file_path_r = os.path.join(base_path, f_name2)

    img_name_l = os.path.join(base_path, img_name1)
    img_name_r = os.path.join(base_path, img_name2)
    
    return text_file_path_l, text_file_path_r, img_name_l, img_name_r

def calculate_distance_from_latlon(lat1, lon1, lat2, lon2):
    R = 6371.0
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c * 1000

    #print('DISTANCE ----', distance)
    return distance 

def calculating_bbx_pixel(x_center, y_center, width, height, img_dir):
    x_min = x_center - width/2
    x_max = x_center + width/2
    
    y_min = y_center - height/2
    y_max = y_center + height/2
    
    im = Image.open(img_dir)
    img_width, img_height = im.size
    
    x_center_p = x_center * img_width
    y_center_p = y_center * img_height
    
    x_min_p = x_min * img_width
    x_max_p = x_max * img_width
    
    y_min_p = y_min * img_height
    y_max_p = y_max * img_height
    
    #print('PIXEL PARAMETERS ----', x_center_p, y_center_p, x_min_p, x_max_p, y_min_p, y_max_p)
    return x_center_p, y_center_p, x_min_p, x_max_p, y_min_p, y_max_p

def get_bbx_param(text_file_path):
    file = open(text_file_path)
    output_lst = file.read().split(' ')
    x_center, y_center, width, height = float(output_lst[1]), float(output_lst[2]), float(output_lst[3]), float(output_lst[4])
    
    #print('ANNOTED PARAMETERS ----', x_center, y_center, width, height)
    return x_center, y_center, width, height

def calculate_depth(baseline, focal_length, disparity):
    depth = baseline * focal_length / disparity
    #print('DEPTH ----', depth)
    return depth

In [3]:
df_req_signs = pd.read_excel('requested_signs10.xlsx')

In [4]:
book2_path = r'D:\Distance Measurement\More New Data\NL-EDEE-150611.DataPackage.csv'
df_book2 = pd.read_csv(book2_path)

In [6]:
df_req_signs.head()

,Sign_name,Latitude,Longitude,Image_Name_1,Image_Name_2,Actual_latitude,Actual_logitude,Actual_left_depth,Actual_right_depth,Predicted_depth
0,blue rectangle direction,52.050723,5.649830,5D3E6DG9_1_L_0_0.jpg,5D3E6DGK_1_L_0_0.jpg,52.050751,5.649743,6.670958,5.801856,7.4136
1,blue circle arrow,52.050589,5.649850,5D3E6DGJ_1_R_0_0.jpg,5D3E6DGU_1_R_0_0.jpg,52.050578,5.649937,6.081167,6.669910,6.4302
2,yellow rectangle,52.051969,5.649560,5D3E6DGZ_1_F_0_0.jpg,5D3E6DH4_1_F_0_0.jpg,52.051963,5.649417,9.796939,11.362659,12.2580
3,blue rectangle with number,52.064448,5.618172,5D3E6DV3_1_L_0_0.jpg,5D3E6DZR_1_L_0_0.jpg,NaN,NaN,NaN,NaN,22.2867
4,blue rectangle with letters,52.058417,5.622334,5D3E6DUE_1_L_0_0.jpg,5D3E6DUF_1_L_0_0.jpg,NaN,NaN,NaN,NaN,5.0920


In [6]:
errors1 = []
errors2 = []
norm_errors1 = []
norm_errors2 = []
yaw_errors1 = []
yaw_errors2 = []

for index, row in df_req_signs.iterrows():
    img1 = row['Image_Name_1'][:8]
    img2 = row['Image_Name_2'][:8]
    lat1, lon1, lat2, lon2 = find_lat_lon(img1, img2)
    car1_car2 = calculate_distance_from_latlon(lat1, lon1, lat2, lon2)
    text_file_path_l, text_file_path_r, img_name_l, img_name_r = get_names(row['Image_Name_1'], row['Image_Name_2'])

    x_center_l, y_center_l, width_l, height_l = get_bbx_param(text_file_path_l)

    x_center_r, y_center_r, width_r, height_r = get_bbx_param(text_file_path_r)

    x_center_p_l, y_center_p_l, x_min_p_l, x_max_p_l, y_min_p_l, y_max_p_l = calculating_bbx_pixel(x_center_l, y_center_l, 
                                                                                                   width_l, height_l, img_name_l)

    x_center_p_r, y_center_p_r, x_min_p_r, x_max_p_r, y_min_p_r, y_max_p_r = calculating_bbx_pixel(x_center_r, y_center_r,
                                                                                                   width_r, height_r, img_name_r)
    
    #disparity = abs(x_center_p_l - x_center_p_r)
    disparity = ( (x_center_p_r - x_center_p_l)**2 + (y_center_p_r - y_center_p_l)**2 )**0.5
    baseline = car1_car2
    focal_length = 331

    depth = calculate_depth(baseline, focal_length, disparity)
    
    Actual_left_depth = row['Actual_left_depth']
    Actual_right_depth = row['Actual_right_depth']
    
    norm_error1 = abs(depth-Actual_left_depth)
    norm_error2 = abs(depth-Actual_right_depth)
    
    if (norm_error1>3) | (norm_error2>3) | (math.isnan(norm_error1)):
        continue
    
    norm_errors1.append(norm_error1)
    norm_errors2.append(norm_error2)
    
    dis = ( (x_center_p_l)**2 + (y_center_p_l)**2 )**0.5
    predicted_left_depth = (depth*dis)/focal_length
    
    dis = ( (x_center_p_r)**2 + (y_center_p_r)**2 )**0.5
    predicted_right_depth = (depth*dis)/focal_length
    
    error1 = abs(predicted_left_depth-Actual_left_depth)
    error2 = abs(predicted_right_depth-Actual_right_depth)
    
    errors1.append(error1)
    errors2.append(error2)
    
    a = df_book2.loc[df_book2['ns1:ImageId'] == img1]
    yaw1 = a.iloc[0]['ns1:RecorderDirection']
    a = df_book2.loc[df_book2['ns1:ImageId'] == img2]
    yaw2 = a.iloc[0]['ns1:RecorderDirection']
    
    ur = x_center_p_r
    vr = y_center_p_r
    b = car1_car2
    d = x_center_p_l - x_center_p_r
    ox = 0
    z = depth
    del_yaw = math.radians(yaw1 - yaw2)
    #del_yaw = yaw1- yaw2

    x2 = b*(ur-ox)/d
    #x2

    corrected_depth = - ( (1+x2) * z * z * del_yaw ) / b 
    depth = depth + corrected_depth
    
    yaw_error1 = abs(depth-Actual_left_depth)
    yaw_error2 = abs(depth-Actual_right_depth)
    
    yaw_errors1.append(yaw_error1)
    yaw_errors2.append(yaw_error2)
    
    #print(lat1, lon1, lat2, lon2)
    #print(img1)

In [7]:
print(errors1)
print(errors2)
print(norm_errors1)
print(norm_errors2)
print(yaw_errors1)
print(yaw_errors2)

[3.6500587249942855, 2.1991152121243633, 8.160043528342104, 1.8535085961320235, 2.4184770489467944, 2.0941057682517235]
[0.21221505807665242, 1.063732329889139, 2.8625046909861016, 2.723195645291799, 1.0714084420044045, 1.0468424758756902]
[0.7238719751347871, 0.34846690607091, 2.294618987804782, 2.5209386733450723, 1.877730253320827, 0.19020088363671306]
[1.592974399930637, 0.24027612248318952, 0.7288987248691914, 0.3234270910458221, 1.0497075209314382, 0.08038650638154277]
[0.41768578644196364, 0.2944446678200485, 3.7074772753531136, 41.54632221055642, 1.3220428551535353, 0.19920689372418554]
[1.2867882112378135, 0.294298360734051, 2.141757012417523, 39.34881062825717, 0.4940201227641463, 0.08939251646901525]


In [ ]:
for i in range(len(errors1)):
    print('1')
    #print(errors1[i])
    print(norm_errors1[i])
    #print(yaw_errors1[i])
    
    print('2')
    #print(errors2[i])
    print(norm_errors2[i])
    #print(yaw_errors2[i])
    
    print('Next')

# Fixing fx

In [8]:
all_fx = []
for index, row in df_req_signs.iterrows():
    img1 = row['Image_Name_1'][:8]
    img2 = row['Image_Name_2'][:8]
    lat1, lon1, lat2, lon2 = find_lat_lon(img1, img2)
    car1_car2 = calculate_distance_from_latlon(lat1, lon1, lat2, lon2)
    text_file_path_l, text_file_path_r, img_name_l, img_name_r = get_names(row['Image_Name_1'], row['Image_Name_2'])

    x_center_l, y_center_l, width_l, height_l = get_bbx_param(text_file_path_l)

    x_center_r, y_center_r, width_r, height_r = get_bbx_param(text_file_path_r)

    x_center_p_l, y_center_p_l, x_min_p_l, x_max_p_l, y_min_p_l, y_max_p_l = calculating_bbx_pixel(x_center_l, y_center_l, 
                                                                                                   width_l, height_l, img_name_l)

    x_center_p_r, y_center_p_r, x_min_p_r, x_max_p_r, y_min_p_r, y_max_p_r = calculating_bbx_pixel(x_center_r, y_center_r,
                                                                                                   width_r, height_r, img_name_r)
    
    #disparity = abs(x_center_p_l - x_center_p_r)
    disparity = ( (x_center_p_r - x_center_p_l)**2 + (y_center_p_r - y_center_p_l)**2 )**0.5
    baseline = car1_car2
    focal_length = 331

    depth = calculate_depth(baseline, focal_length, disparity)
    
    Actual_left_depth = row['Actual_left_depth']
    Actual_right_depth = row['Actual_right_depth']
    
    norm_error1 = abs(depth-Actual_left_depth)
    norm_error2 = abs(depth-Actual_right_depth)
    
    if (norm_error1>3) | (norm_error2>3) | (math.isnan(norm_error1)):
        continue
    
    z1 = Actual_left_depth
    z2 = Actual_right_depth
    d = disparity
    b = baseline
    
    fx = (z1*d)/b
    all_fx.append(fx)
    fx = (z2*d)/b
    all_fx.append(fx)

In [9]:
print(all_fx)

avg_fx = sum(all_fx)/len(all_fx)

avg_focal_length = avg_fx

print(avg_fx)

[298.59876601796145, 259.6968717777211, 313.06079034408583, 343.36950672055764, 268.1860197130236, 311.046783200841, 215.94653991697962, 316.2390673748396, 277.760507292105, 301.23752703178974, 316.8775213074695, 325.0312752399791]
295.58759799477946


In [15]:
errors1 = []
errors2 = []
norm_errors1 = []
norm_errors2 = []
yaw_errors1 = []
yaw_errors2 = []

for index, row in df_req_signs.iterrows():
    img1 = row['Image_Name_1'][:8]
    img2 = row['Image_Name_2'][:8]
    lat1, lon1, lat2, lon2 = find_lat_lon(img1, img2)
    car1_car2 = calculate_distance_from_latlon(lat1, lon1, lat2, lon2)
    text_file_path_l, text_file_path_r, img_name_l, img_name_r = get_names(row['Image_Name_1'], row['Image_Name_2'])

    x_center_l, y_center_l, width_l, height_l = get_bbx_param(text_file_path_l)

    x_center_r, y_center_r, width_r, height_r = get_bbx_param(text_file_path_r)

    x_center_p_l, y_center_p_l, x_min_p_l, x_max_p_l, y_min_p_l, y_max_p_l = calculating_bbx_pixel(x_center_l, y_center_l, 
                                                                                                   width_l, height_l, img_name_l)

    x_center_p_r, y_center_p_r, x_min_p_r, x_max_p_r, y_min_p_r, y_max_p_r = calculating_bbx_pixel(x_center_r, y_center_r,
                                                                                                   width_r, height_r, img_name_r)
    
    #disparity = abs(x_center_p_l - x_center_p_r)
    disparity = ( (x_center_p_r - x_center_p_l)**2 + (y_center_p_r - y_center_p_l)**2 )**0.5
    baseline = car1_car2
    focal_length = avg_focal_length #331

    depth = calculate_depth(baseline, focal_length, disparity)
    
    Actual_left_depth = row['Actual_left_depth']
    Actual_right_depth = row['Actual_right_depth']
    
    norm_error1 = abs(depth-Actual_left_depth)
    norm_error2 = abs(depth-Actual_right_depth)
    
    #if (norm_error1>3) | (norm_error2>3) | (math.isnan(norm_error1)):
        #continue
        
    norm_errors1.append(norm_error1)
    norm_errors2.append(norm_error2)

    dis = ( (x_center_p_l)**2 + (y_center_p_l)**2 )**0.5
    predicted_left_depth = (depth*dis)/focal_length
    
    dis = ( (x_center_p_r)**2 + (y_center_p_r)**2 )**0.5
    predicted_right_depth = (depth*dis)/focal_length
    
    error1 = abs(predicted_left_depth-Actual_left_depth)
    error2 = abs(predicted_right_depth-Actual_right_depth)
    
    errors1.append(error1)
    errors2.append(error2)

    a = df_book2.loc[df_book2['ns1:ImageId'] == img1]
    yaw1 = a.iloc[0]['ns1:RecorderDirection']
    a = df_book2.loc[df_book2['ns1:ImageId'] == img2]
    yaw2 = a.iloc[0]['ns1:RecorderDirection']
    
    ur = x_center_p_r
    vr = y_center_p_r
    b = car1_car2
    d = disparity
    ox = avg_ox
    z = depth
    del_yaw = math.radians(yaw1 - yaw2)
    #del_yaw = yaw1- yaw2

    x2 = b*(ur-ox)/d
    #x2

    corrected_depth = - ( (1+x2) * z * z * del_yaw ) / b 
    depth = depth + corrected_depth
    
    yaw_error1 = abs(depth-Actual_left_depth)
    yaw_error2 = abs(depth-Actual_right_depth)
    
    yaw_errors1.append(yaw_error1)
    yaw_errors2.append(yaw_error2)
    

In [16]:
#print(errors1)
#print(errors2)
print(norm_errors1)
print(norm_errors2)
#print(yaw_errors1)
#print(yaw_errors2)

[0.06727213369749308, 0.3394145781186815, 1.0009902498436745, nan, nan, nan, 8.238787178131243, 11.557964853650315, 5.442449883448514, 1.7450168221761837, 52.982248967218474, 3.4865538915755985, 0.6287525639052962, 0.2867316931250361]
[0.8018302910983568, 0.928157606672781, 0.564730013091916, nan, nan, nan, 3.695575442364433, 7.360256315762655, 9.781103516556895, 0.45249476012306644, 57.85351987660707, 1.4869444019399625, 0.19927016848409274, 0.3965460703802064]


In [17]:
#print(errors1)
#print(errors2)
print(norm_errors1)
print(norm_errors2)
#print(yaw_errors1)
#print(yaw_errors2)

[0.06727213369749308, 0.3394145781186815, 1.0009902498436745, nan, nan, nan, 8.238787178131243, 11.557964853650315, 5.442449883448514, 1.7450168221761837, 52.982248967218474, 3.4865538915755985, 0.6287525639052962, 0.2867316931250361]
[0.8018302910983568, 0.928157606672781, 0.564730013091916, nan, nan, nan, 3.695575442364433, 7.360256315762655, 9.781103516556895, 0.45249476012306644, 57.85351987660707, 1.4869444019399625, 0.19927016848409274, 0.3965460703802064]


# Fixing ox

In [11]:
#ox = ul - (fx*x)/z
all_ox = []
for index, row in df_req_signs.iterrows():
    img1 = row['Image_Name_1'][:8]
    img2 = row['Image_Name_2'][:8]
    lat1, lon1, lat2, lon2 = find_lat_lon(img1, img2)
    car1_car2 = calculate_distance_from_latlon(lat1, lon1, lat2, lon2)
    text_file_path_l, text_file_path_r, img_name_l, img_name_r = get_names(row['Image_Name_1'], row['Image_Name_2'])

    x_center_l, y_center_l, width_l, height_l = get_bbx_param(text_file_path_l)

    x_center_r, y_center_r, width_r, height_r = get_bbx_param(text_file_path_r)

    x_center_p_l, y_center_p_l, x_min_p_l, x_max_p_l, y_min_p_l, y_max_p_l = calculating_bbx_pixel(x_center_l, y_center_l, 
                                                                                                   width_l, height_l, img_name_l)

    x_center_p_r, y_center_p_r, x_min_p_r, x_max_p_r, y_min_p_r, y_max_p_r = calculating_bbx_pixel(x_center_r, y_center_r,
                                                                                                   width_r, height_r, img_name_r)
    
    #disparity = abs(x_center_p_l - x_center_p_r)
    disparity = ( (x_center_p_r - x_center_p_l)**2 + (y_center_p_r - y_center_p_l)**2 )**0.5
    baseline = car1_car2
    focal_length = avg_fx

    depth = calculate_depth(baseline, focal_length, disparity)
    
    Actual_left_depth = row['Actual_left_depth']
    Actual_right_depth = row['Actual_right_depth']
    
    norm_error1 = abs(depth-Actual_left_depth)
    norm_error2 = abs(depth-Actual_right_depth)
    
    if (norm_error1>3) | (norm_error2>3) | (math.isnan(norm_error1)):
        continue
    
    Actual_latitude = row['Actual_latitude']
    Actual_longitude = row['Actual_logitude']
    R = 6371 * 1000 
    #lat1 = math.radians(car1_lat)
    #lon1 = math.radians(car1_lon)
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)

    x11 = R * math.cos(lat1) * math.cos(lon1)
    y11 = R * math.cos(lat1) * math.sin(lon1)
    z11 = R * math.sin(lat1)
    
    print(x11)
    
    x22 = R * math.cos(lat2) * math.cos(lon2)
    y22 = R * math.cos(lat2) * math.sin(lon2)
    z22 = R * math.sin(lat2)

    #x1o, y1o, z1o = x11+x1, y11+y1, z11+z
    #x1o, y1o, z1o = x11+x1, y11+y1, z11+z

    Actual_latitude = math.radians(Actual_latitude)
    Actual_longitude = math.radians(Actual_longitude)
    xo = R * math.cos(Actual_latitude) * math.cos(Actual_longitude)
    yo = R * math.cos(Actual_latitude) * math.sin(Actual_longitude)
    zo = R * math.sin(Actual_latitude)
    
    x1 = (xo - x11)
    y1 = (yo - y11)
    z1 = (zo - z11)
    
    x2 = (xo - x22)
    y2 = (yo - y22)
    z2 = (zo - z22)
    
    ul = x_center_p_l
    ur = x_center_p_r
    b = car1_car2
    d = disparity
    
    ox = ul - (d*x1/b)
    all_ox.append(ox)
    ox = ur - (d*x2/b)
    all_ox.append(ox)
    #lato = math.degrees(math.atan2(y1o, x1o))
    #lono = math.degrees(math.asin(z1o/R))

3898900.506254729
3898912.09772243
3898793.619421212
3895872.5178596997
3898793.619421212
3898873.594907834


In [13]:
print(all_ox)

avg_ox = sum(all_ox)/len(all_ox)

print(avg_ox)

[520.6162493206172, 124.6628307072982, 299.31127640111214, 240.3320867927241, 397.5683117265734, 366.54418396572834, 250.0944036816084, 590.5156793512288, 192.81670467322925, -56.60049994924097, 312.8625765335043, 227.4713162105866]
288.8495932845808


In [18]:
#x1 = (z/fx) * (u-ox)
x_norm_errors = []
for index, row in df_req_signs.iterrows():
    img1 = row['Image_Name_1'][:8]
    img2 = row['Image_Name_2'][:8]
    lat1, lon1, lat2, lon2 = find_lat_lon(img1, img2)
    car1_car2 = calculate_distance_from_latlon(lat1, lon1, lat2, lon2)
    text_file_path_l, text_file_path_r, img_name_l, img_name_r = get_names(row['Image_Name_1'], row['Image_Name_2'])

    x_center_l, y_center_l, width_l, height_l = get_bbx_param(text_file_path_l)

    x_center_r, y_center_r, width_r, height_r = get_bbx_param(text_file_path_r)

    x_center_p_l, y_center_p_l, x_min_p_l, x_max_p_l, y_min_p_l, y_max_p_l = calculating_bbx_pixel(x_center_l, y_center_l, 
                                                                                                   width_l, height_l, img_name_l)

    x_center_p_r, y_center_p_r, x_min_p_r, x_max_p_r, y_min_p_r, y_max_p_r = calculating_bbx_pixel(x_center_r, y_center_r,
                                                                                                   width_r, height_r, img_name_r)
    
    #disparity = abs(x_center_p_l - x_center_p_r)
    disparity = ( (x_center_p_r - x_center_p_l)**2 + (y_center_p_r - y_center_p_l)**2 )**0.5
    baseline = car1_car2
    focal_length = avg_focal_length

    depth = calculate_depth(baseline, focal_length, disparity)
    
    Actual_left_depth = row['Actual_left_depth']
    Actual_right_depth = row['Actual_right_depth']
    
    norm_error1 = abs(depth-Actual_left_depth)
    norm_error2 = abs(depth-Actual_right_depth)
    
    if (norm_error1>3) | (norm_error2>3) | (math.isnan(norm_error1)):
        continue
        
    Actual_latitude = row['Actual_latitude']
    Actual_longitude = row['Actual_logitude']
    R = 6371 * 1000 
    #lat1 = math.radians(car1_lat)
    #lon1 = math.radians(car1_lon)
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)

    x11 = R * math.cos(lat1) * math.cos(lon1)
    y11 = R * math.cos(lat1) * math.sin(lon1)
    z11 = R * math.sin(lat1)
    
    x22 = R * math.cos(lat2) * math.cos(lon2)
    y22 = R * math.cos(lat2) * math.sin(lon2)
    z22 = R * math.sin(lat2)

    #x1o, y1o, z1o = x11+x1, y11+y1, z11+z
    #x1o, y1o, z1o = x11+x1, y11+y1, z11+z

    Actual_latitude = math.radians(Actual_latitude)
    Actual_longitude = math.radians(Actual_longitude)
    xo = R * math.cos(Actual_latitude) * math.cos(Actual_longitude)
    yo = R * math.cos(Actual_latitude) * math.sin(Actual_longitude)
    zo = R * math.sin(Actual_latitude)
    
    x1 = (xo - x11)
    y1 = (yo - y11)
    z1 = (zo - z11)
    
    x2 = (xo - x22)
    y2 = (yo - y22)
    z2 = (zo - z22)
    
    ul = x_center_p_l
    ur = x_center_p_r
    b = car1_car2
    d = disparity
        
    #predicted_x1 = (b/d)*(ul-avg_ox)
    #predicted_x2 = (b/d)*(ur-avg_ox)
    
    predicted_x1 = (depth/avg_fx) * (ul-avg_ox)
    predicted_x2 = (depth/avg_fx) * (ur-avg_ox)
    
    
    x_norm_error = abs(predicted_x1-x1)
    x_norm_errors.append(x_norm_error)
    
    x_norm_error = abs(predicted_x2-x2)
    x_norm_errors.append(x_norm_error)

In [19]:
print(x_norm_errors)

[5.17787029864951, 3.6680762277044128, 0.20321688735657795, 0.9424465013662857, 3.9715368223293748, 2.8382134393041367, 0.8491657373229451, 6.609811667144185, 3.387031902213888, 12.183851837894322, 0.3234057372035206, 0.8266397698123145]


# Fixing oy & fy

In [27]:
all_oy = []
all_fy = []
for index, row in df_req_signs.iterrows():
    img1 = row['Image_Name_1'][:8]
    img2 = row['Image_Name_2'][:8]
    lat1, lon1, lat2, lon2 = find_lat_lon(img1, img2)
    car1_car2 = calculate_distance_from_latlon(lat1, lon1, lat2, lon2)
    text_file_path_l, text_file_path_r, img_name_l, img_name_r = get_names(row['Image_Name_1'], row['Image_Name_2'])

    x_center_l, y_center_l, width_l, height_l = get_bbx_param(text_file_path_l)

    x_center_r, y_center_r, width_r, height_r = get_bbx_param(text_file_path_r)

    x_center_p_l, y_center_p_l, x_min_p_l, x_max_p_l, y_min_p_l, y_max_p_l = calculating_bbx_pixel(x_center_l, y_center_l, 
                                                                                                   width_l, height_l, img_name_l)

    x_center_p_r, y_center_p_r, x_min_p_r, x_max_p_r, y_min_p_r, y_max_p_r = calculating_bbx_pixel(x_center_r, y_center_r,
                                                                                                   width_r, height_r, img_name_r)
    
    #disparity = abs(x_center_p_l - x_center_p_r)
    disparity = ( (x_center_p_r - x_center_p_l)**2 + (y_center_p_r - y_center_p_l)**2 )**0.5
    baseline = car1_car2
    focal_length = avg_focal_length

    depth = calculate_depth(baseline, focal_length, disparity)
    
    Actual_left_depth = row['Actual_left_depth']
    Actual_right_depth = row['Actual_right_depth']
    
    norm_error1 = abs(depth-Actual_left_depth)
    norm_error2 = abs(depth-Actual_right_depth)
    
    if (norm_error1>3) | (norm_error2>3) | (math.isnan(norm_error1)):
        continue
        
    Actual_latitude = row['Actual_latitude']
    Actual_longitude = row['Actual_logitude']
    R = 6371 * 1000 
    #lat1 = math.radians(car1_lat)
    #lon1 = math.radians(car1_lon)
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)

    x11 = R * math.cos(lat1) * math.cos(lon1)
    y11 = R * math.cos(lat1) * math.sin(lon1)
    z11 = R * math.sin(lat1)
    
    x22 = R * math.cos(lat2) * math.cos(lon2)
    y22 = R * math.cos(lat2) * math.sin(lon2)
    z22 = R * math.sin(lat2)

    #x1o, y1o, z1o = x11+x1, y11+y1, z11+z
    #x1o, y1o, z1o = x11+x1, y11+y1, z11+z

    Actual_latitude = math.radians(Actual_latitude)
    Actual_longitude = math.radians(Actual_longitude)
    xo = R * math.cos(Actual_latitude) * math.cos(Actual_longitude)
    yo = R * math.cos(Actual_latitude) * math.sin(Actual_longitude)
    zo = R * math.sin(Actual_latitude)
    
    x1 = (xo - x11)
    y1 = (yo - y11)
    z1 = (zo - z11)
    
    x2 = (xo - x22)
    y2 = (yo - y22)
    z2 = (zo - z22)
    
    vl = y_center_p_l
    vr = y_center_p_r
    b = car1_car2
    d = disparity

    oy, fy = symbols('oy fy')

    eq1 = Eq(oy + (fy*y1/Actual_left_depth) - vl)
    eq2 = Eq(oy + (fy*y2/Actual_right_depth) - vr)
    
    dict_oyfy = solve((eq1,eq2), (oy, fy))
    oy = dict_oyfy.get(oy)
    fy = dict_oyfy.get(fy)
    
    all_oy.append(oy)
    all_fy.append(fy)

In [28]:
all_oy = []
for index, row in df_req_signs.iterrows():
    img1 = row['Image_Name_1'][:8]
    img2 = row['Image_Name_2'][:8]
    lat1, lon1, lat2, lon2 = find_lat_lon(img1, img2)
    car1_car2 = calculate_distance_from_latlon(lat1, lon1, lat2, lon2)
    text_file_path_l, text_file_path_r, img_name_l, img_name_r = get_names(row['Image_Name_1'], row['Image_Name_2'])

    x_center_l, y_center_l, width_l, height_l = get_bbx_param(text_file_path_l)

    x_center_r, y_center_r, width_r, height_r = get_bbx_param(text_file_path_r)

    x_center_p_l, y_center_p_l, x_min_p_l, x_max_p_l, y_min_p_l, y_max_p_l = calculating_bbx_pixel(x_center_l, y_center_l, 
                                                                                                   width_l, height_l, img_name_l)

    x_center_p_r, y_center_p_r, x_min_p_r, x_max_p_r, y_min_p_r, y_max_p_r = calculating_bbx_pixel(x_center_r, y_center_r,
                                                                                                   width_r, height_r, img_name_r)
    
    #disparity = abs(x_center_p_l - x_center_p_r)
    disparity = ( (x_center_p_r - x_center_p_l)**2 + (y_center_p_r - y_center_p_l)**2 )**0.5
    baseline = car1_car2
    focal_length = avg_fx

    depth = calculate_depth(baseline, focal_length, disparity)
    
    Actual_left_depth = row['Actual_left_depth']
    Actual_right_depth = row['Actual_right_depth']
    
    norm_error1 = abs(depth-Actual_left_depth)
    norm_error2 = abs(depth-Actual_right_depth)
    
    if (norm_error1>3) | (norm_error2>3) | (math.isnan(norm_error1)):
        continue
    
    Actual_latitude = row['Actual_latitude']
    Actual_longitude = row['Actual_logitude']
    R = 6371 * 1000 
    #lat1 = math.radians(car1_lat)
    #lon1 = math.radians(car1_lon)
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)

    x11 = R * math.cos(lat1) * math.cos(lon1)
    y11 = R * math.cos(lat1) * math.sin(lon1)
    z11 = R * math.sin(lat1)
    
    x22 = R * math.cos(lat2) * math.cos(lon2)
    y22 = R * math.cos(lat2) * math.sin(lon2)
    z22 = R * math.sin(lat2)

    #x1o, y1o, z1o = x11+x1, y11+y1, z11+z
    #x1o, y1o, z1o = x11+x1, y11+y1, z11+z

    Actual_latitude = math.radians(Actual_latitude)
    Actual_longitude = math.radians(Actual_longitude)
    xo = R * math.cos(Actual_latitude) * math.cos(Actual_longitude)
    yo = R * math.cos(Actual_latitude) * math.sin(Actual_longitude)
    zo = R * math.sin(Actual_latitude)
    
    x1 = (xo - x11)
    y1 = (yo - y11)
    z1 = (zo - z11)
    
    x2 = (xo - x22)
    y2 = (yo - y22)
    z2 = (zo - z22)
    
    ul = x_center_p_l
    ur = x_center_p_r
    b = car1_car2
    d = disparity
    
    oy = vl - (d*y1/b)
    all_oy.append(oy)
    oy = vr - (d*y2/b)
    all_oy.append(oy)

In [29]:
print(all_oy)
print(all_fy)

avg_oy = sum(all_oy)/len(all_oy)
avg_fy = abs(sum(all_fy)/len(all_fy))

print(avg_oy)
print(avg_ox)
print(avg_fx)
print(avg_fy)

[523.1058187374906, 482.1516587339471, -62.40063672439675, -19.673761674840364, 512.6677934287256, 541.6706741386679, 368.23727300128115, 173.49630162787182, 521.9460244628594, 485.7580573647689, -41.96449161881799, 23.795220955953965]
[-1757.68529624378, 15.7236576984943, 664.776377918341, -4.85686297121281, 5.52630003222097, 11.8368920762115]
292.3991610361259
288.8495932845808
295.58759799477946
177.446488581621


In [30]:
y_norm_errors = []
i=0
for index, row in df_req_signs.iterrows():
    i+=1
    img1 = row['Image_Name_1'][:8]
    img2 = row['Image_Name_2'][:8]
    lat1, lon1, lat2, lon2 = find_lat_lon(img1, img2)
    car1_car2 = calculate_distance_from_latlon(lat1, lon1, lat2, lon2)
    text_file_path_l, text_file_path_r, img_name_l, img_name_r = get_names(row['Image_Name_1'], row['Image_Name_2'])

    x_center_l, y_center_l, width_l, height_l = get_bbx_param(text_file_path_l)

    x_center_r, y_center_r, width_r, height_r = get_bbx_param(text_file_path_r)

    x_center_p_l, y_center_p_l, x_min_p_l, x_max_p_l, y_min_p_l, y_max_p_l = calculating_bbx_pixel(x_center_l, y_center_l, 
                                                                                                   width_l, height_l, img_name_l)

    x_center_p_r, y_center_p_r, x_min_p_r, x_max_p_r, y_min_p_r, y_max_p_r = calculating_bbx_pixel(x_center_r, y_center_r,
                                                                                                   width_r, height_r, img_name_r)
    
    #disparity = abs(x_center_p_l - x_center_p_r)
    disparity = ( (x_center_p_r - x_center_p_l)**2 + (y_center_p_r - y_center_p_l)**2 )**0.5
    baseline = car1_car2
    focal_length = avg_focal_length

    depth = calculate_depth(baseline, focal_length, disparity)
    
    Actual_left_depth = row['Actual_left_depth']
    Actual_right_depth = row['Actual_right_depth']
    
    norm_error1 = abs(depth-Actual_left_depth)
    norm_error2 = abs(depth-Actual_right_depth)
    
    if (norm_error1>3) | (norm_error2>3) | (math.isnan(norm_error1)):
        continue
    
    print(i)
        
    Actual_latitude = row['Actual_latitude']
    Actual_longitude = row['Actual_logitude']
    R = 6371 * 1000 
    #lat1 = math.radians(car1_lat)
    #lon1 = math.radians(car1_lon)
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)

    x11 = R * math.cos(lat1) * math.cos(lon1)
    y11 = R * math.cos(lat1) * math.sin(lon1)
    z11 = R * math.sin(lat1)
    
    x22 = R * math.cos(lat2) * math.cos(lon2)
    y22 = R * math.cos(lat2) * math.sin(lon2)
    z22 = R * math.sin(lat2)

    #x1o, y1o, z1o = x11+x1, y11+y1, z11+z
    #x1o, y1o, z1o = x11+x1, y11+y1, z11+z

    Actual_latitude = math.radians(Actual_latitude)
    Actual_longitude = math.radians(Actual_longitude)
    xo = R * math.cos(Actual_latitude) * math.cos(Actual_longitude)
    yo = R * math.cos(Actual_latitude) * math.sin(Actual_longitude)
    zo = R * math.sin(Actual_latitude)
    
    x1 = abs(xo - x11)
    y1 = abs(yo - y11)
    z1 = abs(zo - z11)
    
    x2 = abs(xo - x22)
    y2 = abs(yo - y22)
    z2 = abs(zo - z22)
    
    vl = y_center_p_l
    vr = y_center_p_r
    b = car1_car2
    d = disparity
        
    #predicted_y1 = (b/d)*(vl-avg_oy)
    #predicted_y2 = (b/d)*(vr-avg_oy)
    
    predicted_y1 = (depth/avg_fy) * (vl-avg_oy)
    predicted_y2 = (depth/avg_fy) * (vr-avg_oy)
    
    y_norm_error = abs(predicted_y1-y1)
    y_norm_errors.append(y_norm_error)
    
    y_norm_error = abs(predicted_y2-y2)
    y_norm_errors.append(y_norm_error)

1
2
3
10
13
14


In [31]:
print(y_norm_errors)

[11.7869199690810, 11.5260145844094, 6.23423895604693, 5.46246314823826, 14.0501068352308, 13.8423414469580, 3.14137664856564, 2.21946063034932, 12.0021786518660, 10.7624769764229, 4.90092475834835, 4.04009025510832]


In [32]:
for index, row in df_req_signs.iterrows():
    
    ox = 0
    oy = 0
    fx = fy = 331
    
    img1 = row['Image_Name_1'][:8]
    img2 = row['Image_Name_2'][:8]
    lat1, lon1, lat2, lon2 = find_lat_lon(img1, img2)
    car1_car2 = calculate_distance_from_latlon(lat1, lon1, lat2, lon2)
    text_file_path_l, text_file_path_r, img_name_l, img_name_r = get_names(row['Image_Name_1'], row['Image_Name_2'])

    x_center_l, y_center_l, width_l, height_l = get_bbx_param(text_file_path_l)

    x_center_r, y_center_r, width_r, height_r = get_bbx_param(text_file_path_r)

    x_center_p_l, y_center_p_l, x_min_p_l, x_max_p_l, y_min_p_l, y_max_p_l = calculating_bbx_pixel(x_center_l, y_center_l, 
                                                                                                   width_l, height_l, img_name_l)

    x_center_p_r, y_center_p_r, x_min_p_r, x_max_p_r, y_min_p_r, y_max_p_r = calculating_bbx_pixel(x_center_r, y_center_r,
                                                                                                   width_r, height_r, img_name_r)
    
    #disparity = abs(x_center_p_l - x_center_p_r)
    disparity = ( (x_center_p_r - x_center_p_l)**2 + (y_center_p_r - y_center_p_l)**2 )**0.5
    baseline = car1_car2
    focal_length = fx

    depth = calculate_depth(baseline, focal_length, disparity)
    
    Actual_left_depth = row['Actual_left_depth']
    Actual_right_depth = row['Actual_right_depth']
    
    norm_error1 = abs(depth-Actual_left_depth)
    norm_error2 = abs(depth-Actual_right_depth)
    
    if (norm_error1>3) | (norm_error2>3) | (math.isnan(norm_error1)):
        continue
        
    Actual_latitude = row['Actual_latitude']
    Actual_longitude = row['Actual_logitude']
    R = 6371 * 1000 
    #lat1 = math.radians(car1_lat)
    #lon1 = math.radians(car1_lon)
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)
    

    x11 = R * math.cos(lat1) * math.cos(lon1)
    y11 = R * math.cos(lat1) * math.sin(lon1)
    z11 = R * math.sin(lat1)
    
    x22 = R * math.cos(lat2) * math.cos(lon2)
    y22 = R * math.cos(lat2) * math.sin(lon2)
    z22 = R * math.sin(lat2)
    
    ul = x_center_p_l
    ur = x_center_p_r
    vl = y_center_p_l
    vr = y_center_p_r
    b = car1_car2
    d = disparity
    
    predicted_x1 = (b/d)*(ul-ox)
    predicted_x2 = (b/d)*(ur-ox)
    predicted_y1 = (depth/fy) * (vl-oy)
    predicted_y2 = (depth/fy) * (vr-oy)
    
    x1o, y1o, z1o = x11+predicted_x1, y11+predicted_y1, z11+depth
    #x1o, y1o, z1o = x11+x1, y11+y1, z11+z

    lono = math.degrees(math.atan2(y1o, x1o))
    lato = math.degrees(math.asin(z1o/R))
    d = calculate_distance_from_latlon(lato, lono, Actual_latitude, Actual_longitude)
    print(d)
    

12.09730555974709
11.664002076130078
26.03848236387962
11.186227262214894
26.58885228334979
9.30318132454875


In [116]:
ox = avg_ox
oy = avg_oy
fx = avg_fx
fy = avg_fx

#print(fy)
i = 0

for index, row in df_req_signs.iterrows():
    i+=1
    
    if i!=14:
        continue
    
    img1 = row['Image_Name_1'][:8]
    img2 = row['Image_Name_2'][:8]

    lat1, lon1, lat2, lon2 = find_lat_lon(img1, img2)
    lat1_d, lon1_d, lat2_d, lon2_d = lat1, lon1, lat2, lon2
    car1_car2 = calculate_distance_from_latlon(lat1, lon1, lat2, lon2)
    text_file_path_l, text_file_path_r, img_name_l, img_name_r = get_names(row['Image_Name_1'], row['Image_Name_2'])

    x_center_l, y_center_l, width_l, height_l = get_bbx_param(text_file_path_l)

    x_center_r, y_center_r, width_r, height_r = get_bbx_param(text_file_path_r)

    x_center_p_l, y_center_p_l, x_min_p_l, x_max_p_l, y_min_p_l, y_max_p_l = calculating_bbx_pixel(x_center_l, y_center_l, 
                                                                                                   width_l, height_l, img_name_l)

    x_center_p_r, y_center_p_r, x_min_p_r, x_max_p_r, y_min_p_r, y_max_p_r = calculating_bbx_pixel(x_center_r, y_center_r,
                                                                                                   width_r, height_r, img_name_r)
    
    disparity = ( (x_center_p_r - x_center_p_l)**2 + (y_center_p_r - y_center_p_l)**2 )**0.5
    baseline = car1_car2
    #print(baseline)
    focal_length = fx

    depth = calculate_depth(baseline, focal_length, disparity)
    
    #print(depth)
    
    Actual_latitude = row['Actual_latitude']
    Actual_longitude = row['Actual_logitude']
    R = 6371 * 1000 
    
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)
    sign_latitude = math.radians(Actual_latitude)
    sign_longitude = math.radians(Actual_longitude)
    
    x11 = R * math.cos(lat1) * math.cos(lon1)
    y11 = R * math.cos(lat1) * math.sin(lon1)
    z11 = R * math.sin(lat1)
    
    #print(x11,y11,z11)
    
    x22 = R * math.cos(lat2) * math.cos(lon2)
    y22 = R * math.cos(lat2) * math.sin(lon2)
    z22 = R * math.sin(lat2)
    
    x_mid = (x11+x22)/2
    y_mid = (y11+y22)/2
    
    ul = x_center_p_l
    ur = x_center_p_r
    vl = y_center_p_l
    vr = y_center_p_r
    b = car1_car2
    d = disparity
    
    predicted_x1 = (b/d)*(ul-ox)
    predicted_x2 = (b/d)*(ur-ox)
    predicted_y1 = (depth/fy) * (vl-oy)
    predicted_y2 = (depth/fy) * (vr-oy)
    
    x1o, y1o = x11+predicted_x1, y11-predicted_y1
    #print(x1o, y1o)
    #print(predicted_x1, predicted_y1)
    #x1o, y1o = x_mid+predicted_x1, y_mid+predicted_y1
    #z1o = (depth**2 - (x1o - x11)**2 - (y1o - y11)**2)**0.5  + z11
    
    lono = math.degrees(math.atan2(y1o, x1o))
    #print(z1o)
    #lato = math.degrees(math.asin(z1o/R))
    
    #lono_ = math.degrees(math.atan2(predicted_y1, predicted_x1))
    #lato_ = math.degrees(math.asin(depth/R))

    xo = R * math.cos(sign_latitude) * math.cos(sign_longitude)
    yo = R * math.cos(sign_latitude) * math.sin(sign_longitude)
    zo = R * math.sin(sign_latitude)
    
   
    
    print(x11, y11, z11)
    print(xo, yo, zo)
    print(x1o, y1o, z1o)
    #print(predicted_x1, predicted_y1, depth)
    #print(predicted_x2, predicted_y2, depth)
    #print(lat1_d, lon1_d, lat2_d, lon2_d)
    #print(Actual_latitude, Actual_longitude)
    #print(lato_, lono_)
    
    #error2 = ( (x1o - xo)**2 + (y1o - yo)**2 + (z1o - zo)**2 )**0.5
    #predicted_car1_obj = ( (x1o - x11)**2 + (y1o - y11)**2 + (z1o - z11)**2 )**0.5
    
    
    #print(error2)
    #print(error)
    #print(car1_obj)
    #print(z1o-z11)
    #print('next')
    #----test
    lato = math.degrees(math.acos(x1o / (R*math.cos(math.radians(lono)))))
    #print(lato, lono)
    print(Actual_latitude, Actual_longitude)
    
    #if not math.isnan(Actual_latitude):
        
        
    alad = decimal.Decimal(str(Actual_latitude)).as_tuple().exponent
    #alad = alad
    alod = decimal.Decimal(str(Actual_longitude)).as_tuple().exponent
    #alod = alod
    lato = round(lato, abs(alad))
    lono = round(lono, abs(alod))

    #print(alad, alod)

    #d = decimal.Decimal('56.4325')
    #print(alad.as_tuple().exponent)
    error = calculate_distance_from_latlon(lato, lono, Actual_latitude, Actual_longitude)
    print(lato, lono)
    print(math.degrees(lat1), math.degrees(lon1))
    print(error)
    

    #break

3898873.594907834 385708.0217973814 5023908.34041104
3898874.7954047676 385711.9276048455 5023907.108880576
3898875.1188105047 385708.61918397946 5023912.7983035855
52.05101765 5.64983258
52.05101769 5.64978397
52.05103566 5.649777467000001
3.3239734329185056


In [91]:
a_float = 3.14159
limited_float = round(a_float, 2)

In [92]:
limited_float

3.14